In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# **Importing Tensorflow and other requirements**

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import MaxPool2D , Conv2D , Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import categorical_crossentropy , Accuracy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# ****Loading Dataset****

In [3]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train.shape

(42000, 785)

In [6]:
test.shape

(28000, 784)

# **Defining Trainiing Labels and Testing Labels**

In [7]:
xtrain = train.drop(['label'],axis = 1).astype('int32')
ytrain = train['label'].astype('float32')
xtest = test.astype('float32')

In [8]:
xtrain.shape

(42000, 784)

In [9]:
ytrain.shape

(42000,)

# **Reshaping and Normalizing**

In [10]:
xtrain = xtrain.values.reshape(-1, 28, 28, 1)
xtest = xtest.values.reshape(-1, 28, 28, 1)
xtest = xtest/255.
xtrain = xtrain/255.

In [11]:
ytrain = to_categorical(ytrain,10)
ytrain.shape

(42000, 10)

# **Code For Model**

In [12]:
model = Sequential([
    Conv2D(32 , (3,3) , activation = 'relu' , input_shape = (28,28,1)), #first layer as convolution , 3x3 kernal , activation function is relu , input shape of image is 28,28,1
    Conv2D(32 , (3,3) , activation = 'relu'),
    MaxPool2D(),
    
    
    Conv2D(64 , (3,3) , activation = 'relu',padding='Same' ),
    Conv2D(64 , (3,3) , activation = 'relu',padding='Same' ),
    MaxPool2D(),
    
    Conv2D(128 , (3,3) , activation = 'relu',padding='Same' ),
    Conv2D(128 , (3,3) , activation = 'relu',padding='Same' ),
    MaxPool2D(),
    
    Conv2D(256 , (3,3) , activation = 'relu',padding='Same' ),
    Conv2D(256 , (3,3) , activation = 'relu',padding='Same' ),
    MaxPool2D(),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                    

In [14]:
model.compile(optimizer = 'Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
history=model.fit(xtrain,ytrain, batch_size=128, epochs=10)

Epoch 1/10
329/329 [==============================] - 15s 10ms/step - loss: 0.2790 - accuracy: 0.9055
Epoch 2/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0607 - accuracy: 0.9822
Epoch 3/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0468 - accuracy: 0.9862
Epoch 4/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0298 - accuracy: 0.9912
Epoch 5/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0287 - accuracy: 0.9910
Epoch 6/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0227 - accuracy: 0.9933
Epoch 7/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0195 - accuracy: 0.9941
Epoch 8/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0197 - accuracy: 0.9943
Epoch 9/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0188 - accuracy: 0.9950
Epoch 10/10
329/329 [==============================] - 3s 9ms/step - loss: 0.0175 - accuracy: 0.99

# **Predictions**

In [16]:
preds = model.predict(xtest)
preds = np.argmax(preds,axis=1)
preds = pd.Series(preds,name='Label')

875/875 [==============================] - 2s 2ms/step


In [17]:
submission = pd.concat([pd.Series(range(1,28001),name = 'ImageId'),preds],axis = 1)
submission.to_csv("submission.csv",index = False)